In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

In [ ]:
TD = pd.read_excel('Tableau_Declarations-Prealables-Embauche.xlsx',header=0)
TD.head()

In [ ]:
TD.index

In [ ]:
mask_CDD = TD['Nature_de_contrat'].str.match("CDD.*")
CDD=TD[mask_CDD]
 
rng = pd.date_range('2000 Jan 1', periods=len(CDD), freq='MS')
CDD.index = rng
CDD


In [ ]:
mask_CDI = TD['Nature_de_contrat'].str.match("CDI.*")
CDI=TD[mask_CDI]
rng = pd.date_range('2000 Jan 1', periods=len(CDI), freq='MS')
CDI.index = rng
CDI.describe()

In [ ]:
ax= CDI.plot(y=["DPAE_brut"],style='-')
CDI.resample('BA').mean().plot(y=["DPAE_brut"],style=':',ax=ax,figsize=(20,8))

CDI.boxplot(column=["DPAE_brut"], by='Annee', figsize=(20,8))
CDI.boxplot(column=["DPAE_brut"], by='Trimestre', figsize=(10,8))


In [ ]:
ax= CDD.plot(y=["DPAE_brut"],style='-')
CDD.resample('BA').mean().plot(y=["DPAE_brut"],style=':',ax=ax,figsize=(20,8))

CDD.boxplot(column=["DPAE_brut"], by='Annee', figsize=(20,8))
CDD.boxplot(column=["DPAE_brut"], by='Trimestre', figsize=(10,8))


In [ ]:
EES = pd.read_excel('Tableau_Emploi-Salarie-Trimestre-National_09032022.xlsx',index_col=0)

In [ ]:
EES.head(5)


In [ ]:
EES_A = EES.replace("(O)",pd.NA)

In [ ]:
EES_A.head()

In [ ]:
EES_A.isna().sum()
EES_NET = EES_A.drop(["idBank","Dernière mise à jour", "Période"],axis=1)

In [ ]:
EES_NET.head(5)

In [ ]:
EES_NET=EES_NET.dropna(axis=0)

In [ ]:
EES_NET.head(5)

In [ ]:
EES_NET.loc['total'] = EES_NET.sum(numeric_only=False, axis=0)
EES_NET.tail(1)

In [ ]:
EES_NET=EES_NET.T.copy()
EES_NET.head(5)

In [ ]:
EES_TOTAL = EES_NET['total']
EES_TOTAL



In [ ]:
plt.figure(figsize=(20, 8))
plt.plot(EES_TOTAL)
plt.xlabel("Date")
plt.ylabel("Sum of all fields")
plt.title("Plot")

plt.show()

In [ ]:
#Selection de 4 secteurs
EES_NET[EES_NET.columns[:4]]

In [ ]:
secteur1 = EES_NET[EES_NET.columns[0]].astype(float)
secteur2 = EES_NET[EES_NET.columns[1]].astype(float)
secteur3 = EES_NET[EES_NET.columns[2]].astype(float)
secteur4 = EES_NET[EES_NET.columns[3]].astype(float)

secteurs=[secteur1,secteur2,secteur3,secteur4]


In [ ]:
#Description des secteurs
for secteur in secteurs :
    print(f"\npour {secteur.name} :\n\n{secteur.describe()}")


In [ ]:
#Plot par secteur
for secteur in secteurs :
    plt.figure(figsize=(20, 8))
    plt.plot(secteur.index, secteur.values)
    plt.xlabel("Date")
    plt.ylabel("Emplois")
    plt.title(secteur.name)
    plt.show()

Question 2 :

In [ ]:
#Saisonalité des CDD
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(CDD["DPAE_brut"], period=12) 

plt.figure(figsize=(25,8))
plt.plot(CDD.index, CDD["DPAE_brut"], 'k')
plt.legend(['Cases'])
plt.figure(figsize=(25,8))
plt.plot(CDD.index,decomposition.trend)
plt.plot(CDD.index,decomposition.seasonal)
plt.plot(CDD.index,decomposition.resid)
plt.legend(['Trend', 'Seasonal', 'Resid'])

In [ ]:
#Saisonalité des CDI
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(CDI["DPAE_brut"], period=12)

plt.figure(figsize=(25,8))
plt.plot(CDI.index, CDI["DPAE_brut"], 'k')
plt.legend(['Cases'])
plt.figure(figsize=(25,8))
plt.plot(CDI.index,decomposition.trend)
plt.plot(CDI.index,decomposition.seasonal)
plt.plot(CDI.index,decomposition.resid)
plt.legend(['Trend', 'Seasonal', 'Resid'])


In [ ]:
for secteur in secteurs :
    decomposition = seasonal_decompose( secteur.values,period=4)
    #Emplois
    plt.figure(figsize=(25,8))
    plt.plot(secteur.index,secteur.values , 'k')
    plt.title("Emplois du secteur "+secteur.name)
    plt.show()

    # trend
    plt.figure(figsize=(25,8))
    plt.plot(secteur.index,decomposition.trend)
    plt.title("Tendance du secteur "+secteur.name)
    plt.show()

    #seasonal + resid
    plt.figure(figsize=(25,8))
    plt.title("Secteur "+secteur.name)
    plt.plot(secteur.index,decomposition.seasonal)
    plt.plot(secteur.index,decomposition.resid)
    plt.legend(['Seasonal', 'Resid'])
    plt.show()

Correlation

In [ ]:
CDD_F=CDD[(CDD.index >= pd.Timestamp('2000')) & (CDD.index < pd.Timestamp('2022'))]
dd=pd.DataFrame(columns=list(['Trimestre',"DPAE_brut"]))
for i in range(4*(2022-2000)):
    s=0
    for j in range(3):
        s+=CDD_F.iloc[i*3+j]["DPAE_brut"]
    s/=3
    a=pd.DataFrame(data={"Trimestre" : [str(CDD_F.iloc[i*3]["Annee"])+"-T"+str(CDD_F.iloc[i*3]["Trimestre"])] ,"DPAE_brut":[s]})
    dd = pd.concat([dd , a])
dd.index = dd["Trimestre"]
dd=dd["DPAE_brut"].astype(float)
dd

In [ ]:
#Ramener les secteurs dur la periode 2000-2021

#ici on commence avec 1970, et pour chaque année en a 4 valeurs , pour arriver au debut des valeurs de 2000 on declade de 4*29
secteur1_c = secteur1 [4*29 + 1 :]
secteur2_c = secteur2 [4*29 + 1 :]
secteur3_c = secteur3 [4*29 + 1 :]
secteur4_c = secteur4 [4*29 + 1 :]

secteur_c=[secteur1_c,secteur2_c,secteur3_c,secteur4_c]

secteur1_c

In [ ]:
######################## Correlation avec CDD ####################################
for secteur in secteur_c:
    print(f"\nCorrelation entre CDD et {secteur.name} :\n\t",
    np.corrcoef(secteur.values,dd.values)[0,1])

In [ ]:
#On deduit un DataFrame de forme adequate
CDI_F=CDI[(CDI.index >= pd.Timestamp('2000')) & (CDI.index < pd.Timestamp('2022'))]
di=pd.DataFrame(columns=list(['Trimestre',"DPAE_brut"]))
for i in range(4*(2022-2000)):
    s=0
    for j in range(3):
        s+=CDI_F.iloc[i*3+j]["DPAE_brut"]
    s/=3
    a=pd.DataFrame(data={"Trimestre" : [str(CDI_F.iloc[i*3]["Annee"])+"-T"+str(CDI_F.iloc[i*3]["Trimestre"])] ,"DPAE_brut":[s]})
    di = pd.concat([di , a])
di.index = di["Trimestre"]
di=di["DPAE_brut"].astype(float)

######################## Correlation avec CDI ####################################
for secteur in secteur_c:
    print(f"\nCorrelation entre CDI et {secteur.name} :\n\t",
    np.corrcoef(secteur.values,di.values)[0,1])

Question 3

In [ ]:
CDD = CDD[["DPAE_brut"]]
# CDD.shape
# CDD.info()
CDD_train = CDD[:212]
CDD_test = CDD[212:]
CDD_test

In [ ]:
CDI = CDI[["DPAE_brut"]]
# CDI.shape
# CDI.info()
CDI
CDI_train = CDI[:212]
CDI_test = CDI[212:]



In [ ]:
secteur_train = []
secteur_test = []
for secteur in secteurs :
    secteur_train.append(secteur[:164])
    secteur_test.append(secteur[164:])

  

In [ ]:
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

fit3 = ExponentialSmoothing(
    CDD_train,
    seasonal_periods = 12,
    trend = "add",
    seasonal = "add",
    use_boxcox = False,
    initialization_method = "estimated",
).fit()

In [ ]:
forecast = fit3.forecast(len(CDD_test))
full_curve = np.concatenate((fit3.fittedvalues, forecast))

plt.figure(figsize=(25,8))

plt.plot(CDD_train.index,CDD_train.values, marker="o", color="black", label='CDD_Train')
plt.plot(CDD_test.index,CDD_test.values, marker="o", color="blue", label='CDD_Test')
plt.plot(CDD.index,full_curve, marker="o", color="orange", label='Forecast')
plt.legend()
plt.show()

In [ ]:
fit3 = ExponentialSmoothing(
    CDI_train,
    seasonal_periods = 12,
    trend = "add",
    seasonal = "add",
    use_boxcox = False,
    initialization_method = "estimated",
).fit()

In [ ]:
forecast = fit3.forecast(len(CDI_test))
full_curve = np.concatenate((fit3.fittedvalues, forecast))

plt.figure(figsize=(25,8))

plt.plot(CDI_train.index,CDI_train.values, marker="o", color="black", label='CDI')
plt.plot(CDI_test.index,CDI_test.values, marker="o", color="blue", label='CDI')
plt.plot(CDI.index,full_curve, marker="o", color="orange", label='Forecast')
plt.legend()
plt.show()

In [ ]:
fit3 = []
for secteur in range(4) :
    fit3.append( Holt(
    secteur_train[secteur],
    initialization_method = "estimated",
    ).fit( smoothing_level=0.2, smoothing_trend=0.2) )
    ''' seasonal_periods = 4,
    trend = "add",
    seasonal = "add",
    use_boxcox = False,'''

In [ ]:
forecast = []
for secteur in range(4) :
    forecast.append(fit3[secteur].forecast(len(secteur_test[secteur])))
    full_curve = np.concatenate((fit3[secteur].fittedvalues, forecast[secteur]))

    plt.figure(figsize=(25,8))

    plt.plot(secteur_train[secteur].index,secteur_train[secteur].values, marker="o", color="black", label='Secteur'+secteurs[secteur].name)
    plt.plot(secteur_test[secteur].index,secteur_test[secteur].values, marker="o", color="blue", label='CDI')
    plt.plot(secteurs[secteur].index,full_curve, marker="o", color="orange", label='Forecast')
    plt.legend()
    plt.show()